In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes
from time import time

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RandomizedSearchCV, ShuffleSplit, train_test_split

properati = pd.read_csv('datos/caba_para_mapa.csv',error_bad_lines=False)

<IPython.core.display.Javascript object>

# RF Clasificador Casas

In [58]:
properati = properati.loc[properati['property_type'] == 'house']
properati.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2751 entries, 4 to 48322
Data columns (total 26 columns):
created_on                    2751 non-null object
property_type                 2751 non-null object
place_name                    2751 non-null object
state_name                    2751 non-null object
lat-lon                       2751 non-null object
lat                           2751 non-null float64
lon                           2751 non-null float64
price                         2751 non-null float64
currency                      2751 non-null object
price_aprox_local_currency    2751 non-null float64
price_aprox_usd               2751 non-null float64
surface_total_in_m2           2751 non-null float64
surface_covered_in_m2         2751 non-null float64
price_usd_per_m2              2751 non-null float64
price_per_m2                  2751 non-null float64
floor                         1 non-null float64
rooms                         1134 non-null float64
expenses         

In [59]:
#paso features con strings a numericos
le_barrio = preprocessing.LabelEncoder()
barrios=properati['place_name']
le_barrio.fit(barrios)
properati['place_name'] = le_barrio.transform(barrios)

le_divisa = preprocessing.LabelEncoder()
divisa=properati['currency']
le_divisa.fit(divisa)
properati['currency'] = le_divisa.transform(divisa)

le_tipo = preprocessing.LabelEncoder()
tipos_prop=properati['property_type']
le_tipo.fit(tipos_prop)
properati['property_type'] = le_tipo.transform(tipos_prop)

###  Agrego features

### Pruebo con np.histogram

In [72]:
cant_buckets = 500

#buckets
cantidad,rango = np.histogram(properati['price_per_m2'], bins=cant_buckets)
print rango
properati['categories_by_price']=pd.cut(properati['price_per_m2'],rango,labels=np.arange(cant_buckets),include_lowest=True)
properati['price_range']=pd.cut(properati['price_per_m2'],rango)

properati.loc[properati['categories_by_price']!=properati['categories_by_price']]
#properati['categories_by_price']=pd.qcut(x=properati['price_per_m2'],q=cant_buckets,labels=np.arange(cant_buckets))

[     0.       197.618    395.236    592.854    790.472    988.09
   1185.708   1383.326   1580.944   1778.562   1976.18    2173.798
   2371.416   2569.034   2766.652   2964.27    3161.888   3359.506
   3557.124   3754.742   3952.36    4149.978   4347.596   4545.214
   4742.832   4940.45    5138.068   5335.686   5533.304   5730.922
   5928.54    6126.158   6323.776   6521.394   6719.012   6916.63
   7114.248   7311.866   7509.484   7707.102   7904.72    8102.338
   8299.956   8497.574   8695.192   8892.81    9090.428   9288.046
   9485.664   9683.282   9880.9    10078.518  10276.136  10473.754
  10671.372  10868.99   11066.608  11264.226  11461.844  11659.462
  11857.08   12054.698  12252.316  12449.934  12647.552  12845.17
  13042.788  13240.406  13438.024  13635.642  13833.26   14030.878
  14228.496  14426.114  14623.732  14821.35   15018.968  15216.586
  15414.204  15611.822  15809.44   16007.058  16204.676  16402.294
  16599.912  16797.53   16995.148  17192.766  17390.384  17588.00

,created_on,property_type,place_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,...,title,dist_a_subte,dist_a_tren,dist_a_univ,dist_a_villa,dist_a_zona_anegada,categories_by_price,price_range,anio,is_new


In [73]:
#lo casteo a float porque si no tira error 
properati['categories_by_price']=properati['categories_by_price'].astype(np.float64) 

properati.loc[:,('created_on')] = pd.to_datetime(properati['created_on'])
properati['anio'] = [x.year for x in properati['created_on'] ]

properati.loc[:,'is_new']=properati.loc[:,'description'].str.lower().str.contains('estrenar')

properati.groupby('categories_by_price').count().sort_values('created_on')

,created_on,property_type,place_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,...,description,title,dist_a_subte,dist_a_tren,dist_a_univ,dist_a_villa,dist_a_zona_anegada,price_range,anio,is_new
categories_by_price,,,,,,,,,,,,,,,,,,,,,
496.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
290.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
396.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
394.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
389.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
388.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
383.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
381.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
378.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [74]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'],properati['dist_a_tren'],properati['dist_a_villa'],\
        properati['dist_a_zona_anegada'],properati['surface_total_in_m2'],\
        properati['surface_covered_in_m2'],properati['lat'],properati['lon'])
y = properati['categories_by_price']

### Busco hiper-parametros con random search

In [56]:
%%notify

rf = RandomForestClassifier(n_jobs=-1)

oob_score=[True,False]
criterion = ['gini','entropy']
max_features =["auto","log2",None]
n_estimators = np.arange(10,100,10)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators,"oob_score":oob_score}

search = RandomizedSearchCV(rf, param_distributions=param_grid ,cv=5,n_iter=5) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

ValueError: Unknown label type: 'continuous'

<IPython.core.display.Javascript object>

In [50]:
mejor_rf = search.best_estimator_
print mejor_rf.feature_importances_ # la importancia de cada feature (cuanta informacion le provee al rf) ordenado tal y cual esta zipeado el X

[ 0.05884464  0.05478336  0.11352793  0.0660662   0.10702107  0.24419678
  0.14619598  0.12720687  0.08215716]


In [52]:
search.score(X_test,y_test) # pruebo el estimador refiteado con el set de test

0.27767695099818512

# RF Regresor

In [62]:
#preparo set de datos
X = zip(properati['dist_a_tren'],\
        properati['surface_total_in_m2'],\
        properati['surface_covered_in_m2'],properati['lat'],properati['lon'],properati['categories_by_price'])
y = properati['price_aprox_local_currency']

In [89]:
%%notify

rf = RandomForestRegressor(n_jobs=-1)

criterion = ['mse','mae']
max_features =["auto","log2",None]
n_estimators = np.arange(10,200,10)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators}

search = RandomizedSearchCV(rf, param_distributions=param_grid ,cv=5,n_iter=50,scoring='neg_mean_squared_error') #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 545.82 segundos para 50 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: -265.163 (std: 10.604)
Promedio validation score: -1813.636 (std: 261.321)
Promedio fit time: 1.249s
Hiper-parametros: {'n_estimators': 180, 'max_features': 'auto', 'criterion': 'mse'}

Puesto: 2
Promedio training score: -272.227 (std: 9.576)
Promedio validation score: -1823.823 (std: 252.920)
Promedio fit time: 1.183s
Hiper-parametros: {'n_estimators': 160, 'max_features': 'auto', 'criterion': 'mse'}

Puesto: 3
Promedio training score: -277.982 (std: 8.447)
Promedio validation score: -1824.464 (std: 232.934)
Promedio fit time: 0.695s
Hiper-parametros: {'n_estimators': 100, 'max_features': None, 'criterion': 'mse'}

Puesto: 4
Promedio training score: -264.030 (std: 7.923)
Promedio validation score: -1825.317 (std: 230.146)
Promedio fit time: 1.019s
Hiper-parametros: {'n_estimators': 150, 'max_features': None, 'criterion': 'mse'}

Puesto: 5
Promedio training score: -264.959 (std:

<IPython.core.display.Javascript object>

### mae tarda muchos mas que mse y log 2 anda mejor (mejor cv scor 0.76 )

### Hago GS en los mejores

In [90]:
%%notify

rf = RandomForestRegressor(n_jobs=-1)

criterion = ['mse']
max_features =['auto']
n_estimators = np.arange(100,200,10)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators}

search = GridSearchCV(rf, param_grid=param_grid ,cv=20,scoring='neg_mean_squared_error') #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 267.32 segundos para 10 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: -244.569 (std: 7.427)
Promedio validation score: -1707.378 (std: 698.259)
Promedio fit time: 1.013s
Hiper-parametros: {'max_features': 'auto', 'n_estimators': 120, 'criterion': 'mse'}

Puesto: 2
Promedio training score: -239.696 (std: 6.653)
Promedio validation score: -1719.079 (std: 668.355)
Promedio fit time: 1.348s
Hiper-parametros: {'max_features': 'auto', 'n_estimators': 180, 'criterion': 'mse'}

Puesto: 3
Promedio training score: -241.541 (std: 5.780)
Promedio validation score: -1722.391 (std: 683.045)
Promedio fit time: 1.041s
Hiper-parametros: {'max_features': 'auto', 'n_estimators': 150, 'criterion': 'mse'}

Puesto: 4
Promedio training score: -241.084 (std: 8.329)
Promedio validation score: -1723.572 (std: 703.979)
Promedio fit time: 1.182s
Hiper-parametros: {'max_features': 'auto', 'n_estimators': 160, 'criterion': 'mse'}

Puesto: 5
Promedio training score: -245.508 (s

<IPython.core.display.Javascript object>

In [91]:
#properati['dist_a_tren'],properati['surface_total_in_m2'],properati['surface_covered_in_m2'],properati['lat'],properati['lon'],properati['categories_by_price']
mejor_rf = search.best_estimator_
print mejor_rf.feature_importances_ # la importancia de cada feature (cuanta informacion le provee al rf) ordenado tal y cual esta zipeado el X

[ 0.04007872  0.02581002  0.1777042   0.03869615  0.08888181  0.34242934
  0.10232099  0.12248069  0.06159808]


In [92]:
search.score(X_test,y_test)

-2127.1788427628435

In [93]:
errores = mejor_rf.predict(X_test)-y_test
print ("Error maximo:{0}\nError minimo:{1}".format( max(abs(errores)),min(abs(errores))))
print(errores)

Error maximo:345.791666667
Error minimo:0.0
28693    -96.283333
31246    -11.783333
17834    104.278214
33664     -5.566667
20385     -1.100000
16765    -94.891667
39968     66.008532
23677    -13.191667
14987     22.991667
28704    -11.769444
18737    -25.697222
36448      8.216667
41230     -1.333333
4030     -16.566667
32541      7.500000
26164    -14.316667
1307     -21.385417
40360     23.983333
7578      -4.941667
40008      6.104762
4685      37.603472
23557     -8.658333
127      -12.866667
9947     -74.607738
2923      25.625000
29961     11.412500
15597     49.633333
27062     15.175000
40281     17.750000
25257    -23.952778
            ...    
36544     51.200000
4971      18.058333
11287     -3.280556
40790     -4.226111
1315      14.158333
19385      3.833333
21920     28.566667
8525       0.000000
38893    -61.877778
17727      6.808333
28057     -0.566667
5460       6.766667
35917      9.841667
31210     -5.766667
9614     -55.750000
12328      8.125000
28338    -32.200

In [107]:
count_max=0
max_error=500
for error in errores:
    if abs(error)>max_error:
        count_max+=1
count_max

0